In [18]:
import os
import openai
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
openai.api_key  = os.environ['OPENAI_API_KEY']
persist_directory = "./db/vectorstore"
llm_name = "gpt-3.5-turbo"

In [8]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [12]:
llm.predict("Hi!")

'Hello! How can I assist you today?'

In [13]:
# Building Prompt
template = """You are a legal assistance chatbot for Indian Laws,
use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use 5 sentences maximum. Keep the answer as concise as possible.
Always say "seek expert Legal Advice for more information and assistance." at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

In [14]:
# Retrieval QA Chain
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [16]:
question = "What should I do if my explicit pictures are leaked online without my consent? What acts are punishable?"
result = qa_chain({"query": question})
result["result"]

'If your explicit pictures are leaked online without your consent, you should immediately report the incident to the police and provide them with all the necessary evidence. This act falls under the offense of voyeurism and dissemination of private images without consent, which is punishable under Section 354C of the Indian Penal Code. Seek expert Legal Advice for more information and assistance.'

In [19]:
# Memory for the chatbot
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [21]:
# Conversational Retrieval Chain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [23]:
question = "What should I do if my explicit pictures are leaked online without my consent? What acts are punishable?"
result = qa({"question": question})
result['answer']

'If your explicit pictures are leaked online without your consent, you should take the following steps:\n\n1. Report the incident to the police: Contact your local law enforcement agency and provide them with all the necessary information and evidence.\n\n2. Preserve evidence: Take screenshots or save any evidence related to the incident, such as messages or posts, before they are deleted.\n\n3. Inform the website or platform: Contact the website or platform where the pictures were posted and request that they be taken down. Provide them with any relevant information or evidence.\n\n4. Seek legal advice: Consult with a lawyer who specializes in cybercrime or privacy laws to understand your legal options and rights.\n\nIn terms of the acts that are punishable, the specific laws may vary depending on the jurisdiction. However, in many countries, the following acts are considered offenses:\n\n1. Non-consensual distribution of intimate images: If someone shares or disseminates explicit pic

In [24]:
question = "Now the website is asking me to pay money to take down the pictures, what should I do?"
result = qa({"question": question})
result['answer']

"If a website is asking you to pay money to take down pictures, you should consider the following steps:\n\n1. Review the website's terms and conditions: Check if there are any provisions regarding the removal of content and if any fees are mentioned. \n\n2. Contact the website: Reach out to the website's support or contact information and inquire about their policy for removing content. Ask if there are any alternatives to paying a fee.\n\n3. Seek legal advice: If the website is demanding payment without any legitimate reason or if you believe your rights are being violated, consult with a lawyer who specializes in internet law or intellectual property rights. They can provide guidance on the best course of action.\n\n4. Report the issue: If the website is engaging in illegal activities or violating any laws, you can report the issue to the appropriate authorities or organizations responsible for regulating online content.\n\nRemember, it is important to approach the situation cautiou